# 08. 함수활용

## - Scope
 - LEGB(Local, Enclosed function local, Global, Built-in)

파이썬은 4개의 스코프(Scope)로 나뉘어서 변수에 접근한다.

Built-in은 우리가 여지껏 사용한 `print()`와 같이 파이썬 기본적으로 내장 되어 있는 모듈이다.

그리고 L -> E -> G -> B 순서로 접근한다.

만일 Local 영역이라면 찾고자 하는 변수가 존재하지 않을 때 E 영역을 찾고 또 없으면 G 영역을 찾고 또 없으면 B 영역을 찾는다.

그리고 끝까지 찾았는데도 없다면 undefined (ReferenceError)가 일어날 것이다.

In [1]:
import builtins
builtins.print("빌트인 함수")
print(dir(builtins))

빌트인 함수
['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'Syst

원칙상 Built-in 모듈을 import하여 print(), len(), zip() 등의 함수를 사용해야 하지만 암묵적으로 import 되어있다.

In [2]:
x = 'global x'
print(x)

global x


위 `x`는 global 스코프이다.

In [3]:
x = 'global x'

def localScope():
    y = 'local y'
    print(y)

localScope()
print(x)

local y
global x


위 `y`는 local 스코프이다.

In [4]:
x = 'global x'

def localScope():
    print(x)

localScope()

global x


localScope 함수안의 `x`를 찾으려 할 때 local 스코프에 없으므로 L->E->G->B 순서로 찾는다.

In [5]:
x = 'global x'

def localScope():
    x = "local x"
    print(x, id(x))

localScope()
print(x, id(x))

local x 2353629260848
global x 2353629331184


local 스코프에서 global 스코프의 `x`를 바꿀려고 시도하였지만 바뀌지 않는다.

global의 x와 local의 x는 주소값이 다른 존재이기 때문이다.

즉, 이름은 같지만 사는 차원의 세계가 다른 격이다.

In [6]:
x = 'global x'

def localScope(x):
    print(x, id(x))

localScope("I want to change the global variable")
print(x, id(x))

I want to change the global variable 2353629424304
global x 2353629414640


함수의 매개변수 또한 local 스코프이고 이전 설명과 같은 이유로 이름만 같은 다른 존재이다.

In [7]:
x = 'global x'

def localScope():
    global x
    x = "local x"
    print(x, id(x))

localScope()
print(x, id(x))

local x 2353629416496
local x 2353629416496


만일 함수 안에서 global 스코프의 변수값을 바꾸고 싶다면 `global` 키워드를 사용하면 된다.

그러나 개인적으로 권장하지 않는다. 이는 프로그램을 복잡하게 만들고 디버깅을 어렵게 할 것이다.

In [8]:
x = 'global x'

def outer():
    x = 'outer x'
    
    def inner():
        x = 'inner x'
        print(x)
    
    inner()
    print(x)
    
outer()
print(x)

inner x
outer x
global x


inner 함수 입장에서 outer 함수는 외부함수(Enclosed function local)이다.

만일 프로그램이 inner 함수에서 실행이 된다면 inner()은 local 스코프가 되고 outer은 Enclosed function local 스코프가 된다.

여기서 찾고자 하는 변수가 없다면 inner()를 감싸는 외부함수 outer()의 스코프를 보게 된다.

만일 프로그램이 outer 함수에서 실행이 된다면 outer()가 local 스코프가 될 것이고 

여기서 찾고자 하는 변수가 없다면 outer()를 감싸는 외부 함수가 더는 없으니 바로 global 스코프를 찾을 것이다.

In [9]:
x = 'global x'

def outer():
    x = 'outer x'
    
    def inner():
        nonlocal x
        x = 'inner x'
        print(x) # inner x
    
    inner()
    print(x) # inner x
    
outer()
print(x) # global x

inner x
inner x
global x


만일 내부함수(Local 스코프)가 외부함수(Enclosed function local 스코프)의 변수에 접근하고 싶다면 `nonlocal` 키워드를 사용하면 된다.

주의해야할 점은 외부함수로 감싸진 내부함수 형태가 아니라면 `nonlocal`은 에러를 일으키게 된다는 점이다.

In [10]:
global_var = "test var"

def scope_test():
    local_var = "hello var"
    nonlocal global_var
    global_var = local_var
    
scope_test()
print(global_var)

SyntaxError: no binding for nonlocal 'global_var' found (Temp/ipykernel_17684/3794460605.py, line 5)

아래 코드는 스코프에 대한 총정리를 보여주는 코드다.

In [11]:
def scope_test():

    spam = "test spam"
    
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    do_local()
    print("After local assignment:", spam) # test spam
    do_nonlocal()
    print("After nonlocal assignment:", spam) # nonlocal spam
    do_global()
    print("After global assignment:", spam) # nonlocal spam

scope_test()
print("In global scope:", spam) # global spam

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


추가로 아래와 같이 함수를 호출할 일은 없겠지만 파이썬의 함수 선언의 특징을 보여주는 코드를 보여주겠다.

아래 코드에서 f 함수가 호출되기 전에 `i` 변수값이 6으로 변해 6으로 출력될 것으로 예상되지만 사실은 아니다.

`arg=i`에서 `i` 값이 결정되는 요인은 `호출된 위치`가 아니라 `선언된 위치`이기 때문이다.

In [12]:
i = 5

def f(arg=i): # 선언된 위치
    print(arg)
    
i = 6
f() # 호출된 위치

5


## - 함수의 매개변수

### * Default Value

함수 매개변수에 기본값을 정할 수 있다. 만일 아무 값이 들어오지 않으면 기본값으로 결정될 것이다.

아래에 있는 `raise` 키워드는 에러를 일부러 일으키겠다는 것이다(우리가 원하는 에러 메시지를 정할 수 있다). 

`ValueError`와 같은 에러 객체는 파이썬에서 이미 있는 Built-in 객체이다.

In [13]:
def ask_ok(prompt, retries=2, reminder="Please try again!"):
    while True:
        ok = input(prompt)
        if ok in ('y','ye','yes'):
            return True
        if ok in ('n','no','nop','nope'):
            return False
        retries = retries - 1
        if retries < 0:
            raise ValueError('invalid user response')
        print(reminder)

In [14]:
ask_ok('are you sure?', reminder="다시 시도해주세요")
print("---end---") # raise로부터 Exception이 발생함으로 no print

are you sure? 


다시 시도해주세요


are you sure? 


다시 시도해주세요


are you sure? 


ValueError: invalid user response

### * 기본값 활용시 주의해야할 점

In [15]:
def f(a, L=[]):
    L.append(a)
    print(id(L))
    return L

print(f(1))
print(f(2))
print(f(3))

def another(L=[]):
    print(id(L))
    return L

print(another())
print("-----")
print(f(4))

2353629160512
[1]
2353629160512
[1, 2]
2353629160512
[1, 2, 3]
2353628566208
[]
-----
2353629160512
[1, 2, 3, 4]


f 함수가 호출될 때마다 `L` 매개변수가 `L=[]`로 초기화되어 각각 `[1]`, `[2]`, `[3]`으로 출력되길 바랬겠지만 아니다.

기본 인자는 최초의 호출 시에만 지정된 값으로 초기화 되고 이후의 호출에는 초기화 되지 않는다.

매개변수의 `L` 주소값이 같기에 mutable한 자료형의 변수를 기본값으로 잡게 된다면 주의해야 한다.

In [16]:
def f(param="기본값"):
    print("주소값 :",id(param))
    
f()
f("hello world")
f()

주소값 : 2353630274256
주소값 : 2353629515504
주소값 : 2353630274256


위의 경우에는 자료형 string의 변수는 immutable 하기에 괜찮다.

 - 연속된 호출 간에 mutable한 기본값이 공유되지 않기를 원한다면 아래와 같이

In [17]:
def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

[1]
[2]
[3]


### * `*arg`와 `**arg`

 - `*arg`는 매개변수를 제한 없이 여러개를 받을 수 있음 : 튜플
 - `**arg`는 딕셔너리를 매개변수로 받음
 - `*arg`는 `**arg` 앞으로 와야 한다

In [18]:
def showArgs(a, *b, **c):
    print(a)
    print("-"*10)
    print(b)
    print("-"*10)
    print(c)

In [19]:
showArgs("안녕하세요" ,10,100,1000, A="mic",B="394")

안녕하세요
----------
(10, 100, 1000)
----------
{'A': 'mic', 'B': '394'}


In [20]:
tupVar = (1,10,100,1000)

dicVar = {
    'key A' : 1515,
    'key B' : 5832
}

showArgs("이건 좀 다른 이야기", tupVar, dicVar)

이건 좀 다른 이야기
----------
((1, 10, 100, 1000), {'key A': 1515, 'key B': 5832})
----------
{}


위의 경우는 tupVar과 dicVar의 요소들이 분해되지 않은 상태로 한번에 집어넣어진 것이다.

그러므로 tupVar과 dicVar은 튜플의 요소로 튜플 매개변수에 각각 넣어진다.

딕셔너리 매개변수는 key-value 형태이어야 하므로 아무 것도 안받은 것으로 간주된다.

In [21]:
showArgs("이렇게 넘겨주면 된다", *tupVar, **dicVar)

이렇게 넘겨주면 된다
----------
(1, 10, 100, 1000)
----------
{'key A': 1515, 'key B': 5832}


튜플 자료구조 변수에는 `*`을, 딕셔너리 자료구조 변수에는 `**`을 붙여주면서 매개변수에 담아주면 각 요소가 분해되어 담아질 것이다.

### * 특수 매개 변수

 - `/` : 위치 전용 인자
 - `*` : 키워드 전용 인자

매개변수에 `/`를 사용하면 `/` 앞에 있는 매개변수는 키워드가 아닌 위치 매개변수로서만 작동한다.

매개변수에 `*`를 사용하면 `*` 뒤에 있는 매개변수는 키워드 매개변수로서만 작동한다.

둘 다 사용하게 된다면 `/`이 먼저 나와야하고 그 다음 `*`가 나와야 한다

In [22]:
def standard_arg(arg):
    print("standard arg : ", arg)

def pos_only_arg(arg, /):
    print("pos only arg : ", arg)
    
def kwd_only_arg(*, arg):
    print("kwd only arg: ", arg)

def combined_example(pos_only, /, standard, *, kwd_only):
    print(pos_only, standard, kwd_only)

In [23]:
standard_arg(10)
standard_arg(arg=20)

standard arg :  10
standard arg :  20


In [24]:
pos_only_arg(30)
pos_only_arg(arg=40)

pos only arg :  30


TypeError: pos_only_arg() got some positional-only arguments passed as keyword arguments: 'arg'

In [25]:
kwd_only_arg(arg=50)
kwd_only_arg(60)

kwd only arg:  50


TypeError: kwd_only_arg() takes 0 positional arguments but 1 was given

In [26]:
combined_example(1, 2, kwd_only=3)

1 2 3


 - 딕셔너리를 통한 특수 매개변수 사용의 필요성

아래와 같은 논리를 가지고 있는 함수를 보자

In [27]:
# member={
#     'name':'James',
#     'age':10,
#     'phone':'010-1234-5678'
# }
member=dict(
    name='James',
    age=10,
    phone='010-1234-5678'
)

print('name' in member)
print('password' in member)

True
False


In [28]:
def foo(key, **kwds):
    return key in kwds

In [29]:
foo('name', **member)

True

딕셔너리의 키(name)와 함수의 매개변수(name)이 충돌이 일어날 수도 있다.

매개변수에 name이라는 이름이 중복되기 때문이다.

```python
name='name', name='James'...
```

In [30]:
def foo(name, **kwds):
    return name in kwds

In [31]:
foo('name', **member)

TypeError: foo() got multiple values for argument 'name'

그러므로 딕셔너리의 키이름과의 충돌을 피하기위해 특수 매개 변수를 사용할 필요가 있다.

In [32]:
def foo(name, /, **kwds):
    return name in kwds

In [33]:
foo('name',**member)

True

 - 특수 매개변수를 이용해 아래와 같이 사용할 수도 있다.

In [34]:
args = (3, 6)
list(range(*args))

[3, 4, 5]

## - Documentation

협업을 할 때 모두를 위해 함수에 대한 설명서를 적어두는 것이 좋을 것이다. 

설사 자신 작성한 함수더라도 긴 시간이 지나면 자신도 어떤 함수였는지 잊어버리게 될 것이다.

In [35]:
def my_function():
    """\
    이 my_function 함수는 어떻게 Documentation을 하는지
    알리기 위해서 사용되어졌습니다.
    
    오예~
    """
    pass

print(my_function.__doc__)

    이 my_function 함수는 어떻게 Documentation을 하는지
    알리기 위해서 사용되어졌습니다.
    
    오예~
    


`"""\`에 `\`를 추가한 이유는 개행을 막기 위해서이다. 만일 `\`가 없게 되면 개행이 된 상태로 출력될 것이다.

In [36]:
def my_function():
    """
    이 my_function 함수는 어떻게 Documentation을 하는지
    알리기 위해서 사용되어졌습니다.
    
    오예~
    """
    pass

print(my_function.__doc__)


    이 my_function 함수는 어떻게 Documentation을 하는지
    알리기 위해서 사용되어졌습니다.
    
    오예~
    


## - Annotation
 - 어노테이션은 메타데이터 정보
 - `__annotation__`에 dict로 저장됨
 - 매개변수 뒤 `:`으로 정의됨
 - Java와 달리 Python에서의 어노테이션은 사용자에게 인자값의 형태와 반환값을 알려주는 것

파이썬의 변수는 할당 값에 따라 자료형이 바뀌게 된다.

즉, 함수의 매개변수에 문자열이 들어올 수도, 리스트가 들어올 수도, 정수가 들어올 수도 있다는 말이다.

비록 어노테이션이 특정 자료형 사용을 강제하지 않지만 함수를 사용하는 개발자에게 좋은 힌트가 될 것이다.

개인적으로 협업시 함수를 만들 때 `Documentation`과 `Annotation`을 습관적으로 작성하는 것이 좋다고 생각한다.

In [37]:
score: int = 91
__annotations__

{'score': int}

In [38]:
def f(ham: str, eggs: str = 'eggs') -> int: # 이 함수는 ham과 eggs 매개변수에는 문자열이 들어오고 정수가 반환됩니다.
    print("Annotations:", f.__annotations__)
    print("Arguments:", ham, eggs)
    return 0
f('spam')

Annotations: {'ham': <class 'str'>, 'eggs': <class 'str'>, 'return': <class 'int'>}
Arguments: spam eggs


0

## - `map`, `filter`, `reduce`

위 함수를 통해 가독성이 좋은 코드를 만들 수 있다.

아래 코드를 보자

In [39]:
# datas 요소에 10을 더하고 그 결과가 짝수인 요소들만 골라 모두 더하여라
datas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# 모든 요소에 10을 더하고
def plusten(L: list) -> None:
    for i in range(len(L)):
        L[i] += 10

# 그 결과가 짝수인 요소들만 모두 골라
def selectEven(L: list) -> list:
    newL = []
    for i in range(len(L)):
        if L[i] % 2 == 0:
            newL.append(L[i])
    return newL
        
# 모두 더하여라
def addAll(L: list) -> int:
    result = 0
    for element in L:
        result += element
    return result

plusten(datas)
print(datas)
filtered_datas = selectEven(datas)
print(filtered_datas)
result = addAll(filtered_datas)
print(result)

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[12, 14, 16, 18, 20]
80


코드가 생각보다 긴 것을 볼 수 있다. 

각 과정을 1줄씩 표현하면 가독성이 좋지 않을까?

### * map

map 함수는 모든 요소에 대해 주어진 함수를 실행한다.



### * filter

filter 함수는 주어진 함수에 대하여 true인 요소만을 가지고 나머지는 무시한다.

### * reduce

reduce 함수는 요소를 줄여가면서 주어진 함수를 실행한다.

Built-in 함수가 아니라서 functools라는 내부 모듈을 불러와야 한다

예를 들어 아래 코드가 있다고 가정해 보자
```python
reduce(lambda x, y : x + y, [1,2,3,4,5])
```
이 코드는 아래와 같이 동작한다.

`step 1`
```
x = 1, y = 2, x + y은 3
```

`step 2` : step 1의 결과가 x의 매개변수로 들어간다.
```
x = 3, y = 3, x + y은 6 
```

`step 3` : step 2의 결과가 x의 매개변수로 들어간다.
```
x = 6, y = 4, x + y은 10
```

`step 4` : step 3의 결과가 x의 매개변수로 들어간다.
```
x = 10, y = 5, x + y은 15
```

더 이상 수행할 요소가 없으므로 최종 결과값인 15가 반환된다.

In [40]:
from functools import reduce

# datas 요소에 10을 더하고 그 결과가 짝수인 요소들만 골라 모두 더하여라
datas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

datas = map(lambda x : x + 10, datas) # plusten
datas = filter(lambda x : x % 2 == 0, datas) # selectEven
datas = reduce(lambda x, y : x + y, datas) # addAll
print("result : ", datas)

result :  80


위 동작을 한줄로 표현하면 다음과 같다.

In [41]:
from functools import reduce

# datas 요소에 10을 더하고 그 결과가 짝수인 요소들만 골라 모두 더하여라
datas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

datas = reduce(lambda x, y : x + y, filter(lambda x : x % 2 == 0, map(lambda x : x + 10, datas)))
print("result : ", datas)

result :  80


<br>
<br>
<br>
<br>
<br>
<br>
<hr>
<br>
<br>
<br>
<br>
<br>
<br>